In [7]:
%pip install groq

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
os.environ['GROQ_API_KEY'] = "Sua chave"

In [9]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Fast language models are essential for various applications and have become increasingly important in recent years. Here are some reasons why fast language models matter:

1. **Real-time Applications**: Fast language models enable real-time applications such as chatbots, virtual assistants, and language translation software. These models can process and respond to user input quickly, providing a seamless and interactive experience.
2. **Improved User Experience**: Fast language models reduce latency, which is critical for providing a good user experience. When models respond quickly, users are more likely to engage with them, and the overall interaction becomes more efficient and enjoyable.
3. **Scalability**: Fast language models can handle a large volume of requests simultaneously, making them suitable for large-scale applications such as customer service chatbots, language translation platforms, and text summarization tools.
4. **Enhanced Productivity**: Fast language models can aut

Nesse trecho é feito o import de bibliotecas e a criação de um objeto cliente que vai se comunicar com a API da Groq, também é feito um teste de requisição para o modelo utilizado.

In [10]:
class Agent:
    def __init__(self, client,  system):
        self.client = client
        self.system = system
        self.messages = []
        if self.system is not None:
            self.messages.append({"role": "system", "content": self.system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        completion = client.chat.completions.create(
            messages=self.messages,
            model="llama-3.3-70b-versatile",
        )
        return completion.choices[0].message.content

Aqui é criado a classe Agent, onde é implementado o construtor e dois métodos que são essenciais para o funcionamento do agente. O construtor (__init__) é executado assim que o objeto Agent é criado, definindo o client e salvando a mensagem de sistema, caso haja uma, na lista de mensagens que acabou de ser criada. Já o método __call__ é o responsável por permitir que o Agent possa ser chamado, mesmo que sem uma mensagem, e também chama o método de execução. Por último, o método "execute" é o responsável por enviar a requisição para o modelo e retornar a resposta recebida do modelo.

In [11]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE 

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944x10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944x10e25.

Now it's your turn:
""".strip()

In [12]:
#tools
def calculate(operation):
    return eval(operation)

def get_planet_mass(planet) -> float:
    if planet == "earth":
        return 5.972e24
    if planet == "mars":
        return 6.39e23
    if planet == "jupiter":
        return 1.898e27
    if planet == "saturn":
        return 5.683e26
    if planet == "uranus":
        return 8.681e25
    if planet == "neptune":
        return 1.024e26
    if planet == "mercury":
        return 3.285e23
    if planet == "venus":
        return 4.867e24
    return None

Aqui é feita a implementação das funções(tools) que o agente pode utilizar para resolver o problema apresentado.

In [14]:
neil_tyson = Agent(client, system_prompt)

Aqui apenas é criado o agente passando o prompt

In [15]:
result = neil_tyson("What is the mass of earth times 5?")
print(result)

Thought: To find the mass of Earth times 5, I first need to know the mass of Earth. 

Action: get_planet_mass: Earth
PAUSE


Aqui é demonstrado um dos príncipios da abordagem baseada em agentes, o de quebrar o problema em partes menores, além de raciocinar sobre como resolvê-lo e identificando a necessidade de obter informações externas

In [16]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944x10e25\n\nIf you have the

In [17]:
result = neil_tyson()
print(result)

In [19]:
observation = get_planet_mass("earth")
print(observation)

5.972e+24


In [20]:
next_prompt = f"Observation: {observation}"
result = neil_tyson(next_prompt)
print(result)

Thought: Now that I have the mass of Earth, I can multiply it by 5 to get the final result.

Action: calculate: 5.972e24 * 5
PAUSE


In [21]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944x10e25\n\nIf you have the

In [22]:
observation = calculate("3.285e23 * 5")
print(observation)

1.6425e+24


In [23]:
next_prompt = f"Observation: {observation}"
result = neil_tyson(next_prompt)
print(result)

Thought: I have calculated the mass of Earth times 5, so now I can provide the answer.

Answer: The mass of Earth times 5 is 2.986e+25


In [24]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE \n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this: \n\nObservation: 1,1944x10e25\n\nIf you have the

In [27]:
import re

def agent_loop(max_iterations, system, query):
    agent = Agent(client, system_prompt)
    tools = ['calculate', 'get_planet_mass']
    next_prompt = query
    i = 0
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"
            
            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        if "Answer" in result:
            break


Esse bloco de código define como será o loop de funcionamento do agente. Nele, inicialmente é instanciado o agent e definindo quais ferramentas ele poderá usar. Dentro do loop, é usada uma estrutura condicional para analisar o resultado gerado pelo modelo e identificar se o agente precisará usar alguma tool e, caso precise, identificando qual será a tool utilizada. O ciclo continua até que o modelo gere uma resposta final, sendo esse o ponto de parada.

In [28]:
agent_loop(max_iterations=10, system=system_prompt, query="What is the mass of the earth plus the mass of mercury, and all of it times 5?")

Thought: To solve this problem, I first need to find the masses of Earth and Mercury. Then, I'll add these masses together and multiply the sum by 5.

Action: get_planet_mass: Earth
PAUSE>NNRecognitionException
Observation: None
Thought: It seems the observation for the mass of Earth was not provided. I'll try again to get the mass of Earth and then proceed with getting the mass of Mercury.

Action: get_planet_mass: Earth
PAUSE
Observation: None
Thought: I still didn't receive the mass of Earth. Let me try to get the mass of Mercury instead, and then come back to Earth's mass.

Action: get_planet_mass: Mercury
PAUSE
Observation: None
Thought: Unfortunately, I didn't receive the mass of Mercury either. Let me try again to get the mass of Earth, as it's essential for the calculation.

Action: get_planet_mass: Earth
PAUSE
Observation: None
Thought: Since I'm having trouble getting the masses directly, I'll recall that the mass of Earth is approximately 5.972e24 kg and the mass of Mercury 